In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Rescaling, Cropping2D, concatenate
from keras.models import Sequential, load_model
from keras import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import matplotlib.pyplot as plt
import math
import zipfile
import h5py
import os

In [9]:
model_name = 'Multicamera_15'
version = 'version_1'
dataset_path = f'/tf/datasets/{model_name.lower()}.h5'
base_dir = '/tf/Models'
work_dir = os.path.join(base_dir, model_name, version)

data = h5py.File(dataset_path,  'r')

In [10]:
checkpoint_path = os.path.join(work_dir, 'weights_improvement_{epoch:02d}_{val_loss:.4f}.hdf5')
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto')
callbacks = [checkpoint, early_stop]

In [11]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, file, list_IDs, batch_size=32):
        'Initialization'
        self.batch_size = batch_size
        self.file = file
        self.list_IDs = list_IDs

    def __len__(self):
        'Denotes the number of batches per epoch'   
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch

        indexes = self.list_IDs[index*self.batch_size:(index+1)*self.batch_size]

        IMAGES = self.file['images'][indexes]
        INPUT_2 = self.file['input_2'][indexes]
        LABELS = self.file['labels'][indexes]

        return [IMAGES, INPUT_2], LABELS

In [12]:
def create_model():
    img_inputs = keras.Input(shape=(180, 300, 3))
    cropping = Cropping2D(cropping=((70, 0), (0, 0)))
    rescaling = Rescaling(scale=1. / 255, offset=-0.5)
    x = cropping(img_inputs)
    x = Rescaling(scale=1. / 255, offset=-0.5)(x)
    x = Conv2D(24, (5, 5), strides=(2, 2), activation="relu")(x)
    x = Conv2D(36, (5, 5), strides=(2, 2), activation="relu")(x)
    x = Conv2D(48, (5, 5), strides=(2, 2), activation="relu")(x)
    x = Conv2D(64, (3, 3), strides=(2, 2), activation="relu")(x)
    x = Conv2D(64, (3, 3), strides=(1, 1), activation="relu")(x)
    x = Conv2D(64, (3, 3), strides=(1, 1), activation="relu")(x)
    x = Flatten()(x)
    x = Activation("relu")(x)
    x = Dense(100)(x)
    x = Model(inputs=img_inputs, outputs=x)
    hlc_inputs = keras.Input(shape=(6))
    combined = concatenate([x.output, hlc_inputs])
    z = Dense(100, activation="relu")(combined)
    z = Dense(50, activation="relu")(z)
    z = Dense(10, activation="relu")(z)
    output = Dense(3)(z)
    model = Model(inputs=[x.input, hlc_inputs], outputs=output)
    # keras.utils.plot_model(model, show_shapes=True, dpi=50)
    model.compile(optimizer='Adam', loss='mse')
    return model

model = create_model()

In [13]:
n = data['images'].shape[0]
training = math.floor(n*0.80)
list_IDs_train = np.arange(training)
list_IDs_val = np.arange(training, n)

In [ ]:
history = model.fit(
    DataGenerator(file=data, list_IDs=list_IDs_train),
    validation_data=DataGenerator(file=data, list_IDs=list_IDs_val),
    epochs=100,
    callbacks=callbacks,
)

Epoch 1/100
13497/13500 [============================>.] - ETA: 0s - loss: 0.0219

In [9]:
np.save(os.path.join(work_dir, f'{model_name.lower()}_history.npy'), history.history)

In [10]:
model.save(os.path.join(work_dir, f'{model_name.lower()}_model'))

INFO:tensorflow:Assets written to: /tf/Models/Multicamera_7/version_1/multicamera_7_model/assets


INFO:tensorflow:Assets written to: /tf/Models/Multicamera_7/version_1/multicamera_7_model/assets
